# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy


In [2]:
path = r'/Users/gil/Documents/06-2023 Instacart Basket Analysis New'


# Import dataframes

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data','orders_products_merged_updated_2.pkl'))


In [7]:
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data','customers.csv'), index_col = False)


# Data wrangling

In [8]:
customers

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
...,...,...,...,...,...,...,...,...,...,...
206204,168073,Lisa,Case,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,4/1/2020,1,married,99799


## Changing columns' names

In [12]:
#change column Surnam to Last Name
customers.rename(columns = {'Surnam' : 'Last Name'}, inplace = True)


In [13]:
#change n_dependants to Number of Dependents 
customers.rename(columns = {'n_dependants' : 'Number of Dependents'}, inplace = True)


In [14]:
#change fam_status to Marital Status 
customers.rename(columns = {'fam_status' : 'Marital Status'}, inplace = True)

## Changing data types

In [15]:
customers.describe()

,user_id,Age,Number of Dependents,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [16]:
customers['user_id'] = customers['user_id'].astype('str')



In [17]:
customers.describe()

,Age,Number of Dependents,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


# Data consistency

In [20]:
#check for mixed data
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)


First Name


In [21]:
#change data type to be all string
customers['First Name'] = customers['First Name'].astype('str')


In [22]:
#confirm that it was fixed
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

In [23]:
customers.isnull().sum()

user_id                 0
First Name              0
Last Name               0
Gender                  0
STATE                   0
Age                     0
date_joined             0
Number of Dependents    0
Marital Status          0
income                  0
dtype: int64

### There are no missing values

In [25]:
df_dupes = customers[customers.duplicated()]

In [26]:
df_dupes

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,Number of Dependents,Marital Status,income


### There are no duplicates

# Combine customer data with the rest of the data

In [28]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,number_of_days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,...,price_range_loc,busiest_day,busiest_days,busiest_time_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_prior_orders,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,195,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,14.0,High spender,30.0,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,195,...,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,14.0,High spender,30.0,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,195,...,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,14.0,High spender,30.0,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,195,...,Mid-range product,Least busy,Least busy,Average orders,10,New customer,14.0,High spender,30.0,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,195,...,Mid-range product,Least busy,Least busy,Most orders,10,New customer,14.0,High spender,30.0,Non-frequent customer


In [30]:
#check what data type is user_id - the key column
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 25 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   order_id                          int64   
 1   user_id                           int64   
 2   order_number                      int64   
 3   orders_day_of_week                int64   
 4   order_hour_of_day                 int64   
 5   number_of_days_since_prior_order  float64 
 6   product_id                        int64   
 7   add_to_cart_order                 int64   
 8   reordered                         int64   
 9   Unnamed: 0_y                      int64   
 10  product_name                      object  
 11  aisle_id                          int64   
 12  department_id                     int64   
 13  prices                            float64 
 14  _merge                            category
 15  price_range_loc                   object  
 16  busiest_day     

In [32]:
#check what data type is user_id - the key column
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_id               206209 non-null  object
 1   First Name            206209 non-null  object
 2   Last Name             206209 non-null  object
 3   Gender                206209 non-null  object
 4   STATE                 206209 non-null  object
 5   Age                   206209 non-null  int64 
 6   date_joined           206209 non-null  object
 7   Number of Dependents  206209 non-null  int64 
 8   Marital Status        206209 non-null  object
 9   income                206209 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 15.7+ MB


In [33]:
#since the key column is of a different type in both data sets, we need to change it to string in ords_prods_merge
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')


In [34]:
#confirm that user_id was changed to object
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 25 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   order_id                          int64   
 1   user_id                           object  
 2   order_number                      int64   
 3   orders_day_of_week                int64   
 4   order_hour_of_day                 int64   
 5   number_of_days_since_prior_order  float64 
 6   product_id                        int64   
 7   add_to_cart_order                 int64   
 8   reordered                         int64   
 9   Unnamed: 0_y                      int64   
 10  product_name                      object  
 11  aisle_id                          int64   
 12  department_id                     int64   
 13  prices                            float64 
 14  _merge                            category
 15  price_range_loc                   object  
 16  busiest_day     

In [35]:
#Drop the Merge column
ords_prods_merge = ords_prods_merge.drop(columns = ['_merge'])


In [36]:
#merge the datasets
customers_ords_prods_merged = ords_prods_merge.merge(customers, on = 'user_id', indicator = True)


In [37]:
#check the values
customers_ords_prods_merged['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [38]:
customers_ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,number_of_days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,...,First Name,Last Name,Gender,STATE,Age,date_joined,Number of Dependents,Marital Status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [39]:
customers_ords_prods_merged.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'number_of_days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'Unnamed: 0_y', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_time_of_day', 'max_order', 'loyalty_flag',
       'avg_price', 'spending_flag', 'median_prior_orders',
       'order_frequency_flag', 'First Name', 'Last Name', 'Gender', 'STATE',
       'Age', 'date_joined', 'Number of Dependents', 'Marital Status',
       'income', '_merge'],
      dtype='object')

In [40]:
#drop the unnecessary Unnamed:0_y column 
customers_ords_prods_merged = customers_rds_prods_merged.drop(columns = ['Unnamed: 0_y'])

In [43]:
customers_ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,number_of_days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,First Name,Last Name,Gender,STATE,Age,date_joined,Number of Dependents,Marital Status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [44]:
#export the dataset
customers_ords_prods_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'customers_ords_prods_merged.pkl'))
